In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import decomposition
from xgboost import XGBClassifier
import pickle

In [0]:
def datarize(path):
  df_rows = []
  with open(path, encoding="utf-8") as f:
    for line in f.readlines():
      line = line.strip()
      features = line.split("\x01")
      df_rows.append(features)

  df = pd.DataFrame(df_rows)
  columns = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
              "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
              "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
              "enaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
              "enaging_user_account_creation", "engagee_follows_engager", "reply", "retweet", "retweet_with_comment", "like"]
  df.columns = columns
  return df

In [1]:
!head 10 '/content/drive/My Drive/網路搜索與探勘/recsys challenge 2020/validation data/val.tsv'

head: cannot open '10' for reading: No such file or directory
==> /content/drive/My Drive/網路搜索與探勘/recsys challenge 2020/validation data/val.tsv <==
101	47185	10157	100986	10343	55422	119	14120	131	120	120	188	119	11170	120	162	10162	10575	10237	11281	10123	11305	11537	10350	10115	1027647B4E9DAF4C1D8973397DC2A04F3E3PhotoTopLevelD3164C7FBCF2565DDF915B1B3AEFB1DC15817031268A9AB92B775C62C4AB60DF6773A01571139411216false14482921860000006C3074607050F1339DDCB890BB27448600false1520948869true
101	56898	137	10730	107105	59578	17394	17394	17394	131	14120	131	120	120	188	119	11170	120	150	10305	11396	10731	11373	10575	10457	11127	10707	102408DB1803264B5FF55F73EC06BE9BD77Photo	Photo	Photo	PhotoRetweetB9175601E87101A984A50F8A62A1C3741582021842825B003667DF15692B1704F44DAFD1A2186100false1263078566000013315386492275CCBF7AEF293EF0139956false1335110299false
101	16735	11584	131	14120	131	120	120	188	119	11170	120	10126	11127	30331	24323	10731	10115	11537	1022EE951379C

## 1GB data

In [0]:
df = datarize('/content/drive/My Drive/網路搜索與探勘/recsys challenge 2020/training data/Train.tsv.partae')

In [0]:
df = df[1:500001]

In [0]:
df.to_csv('/content/drive/My Drive/網路搜索與探勘/recsys challenge 2020/training data/50w_data.csv')

In [0]:
text_train  = df['text_tokens']
hashtags_train  = df['hashtags']

In [0]:
def encoding(df):
  df.loc[df['reply']!='', 'reply'] = 1
  df.loc[df['reply']!=1, 'reply'] = 0
  df.loc[df['retweet']!='', 'retweet'] = 1
  df.loc[df['retweet']!=1, 'retweet'] = 0
  df.loc[df['retweet_with_comment']!='', 'retweet_with_comment'] = 1
  df.loc[df['retweet_with_comment']!=1, 'retweet_with_comment'] = 0
  df.loc[df['like']!='', 'like'] = 1
  df.loc[df['like']!=1, 'like'] = 0

  df.loc[(df['present_media']!='Photo') & (df['present_media']!='Video'), 'present_media']='others'      
  df.loc[(df['tweet_type']!='Top_Level') & (df['tweet_type']!='Retweet') & (df['tweet_type']!='Quote'), 'tweet_type'] = 'Top_Level'   
  df.loc[(df['language']!='D3164C7FBCF2565DDF915B1B3AEFB1DC') & (df['language']!='22C448FF81263D4BAF2A176145EE9EAD') & (df['language']!='06D61DCBBE938971E1EA0C38BD9B5446') & (df['language']!='ECED8A16BE2A5E8871FD55F4842F16B1') & (df['language']!='B9175601E87101A984A50F8A62A1C374'), 'language'] = 'others'
  df = df.drop(columns=['text_tokens', 'hashtags', 'tweet_id', 'present_links', 'present_domains', 'tweet_timestamp', 'enaging_user_id', 'engaged_with_user_account_creation', 'engaged_with_user_id', 'enaging_user_account_creation', "reply", "retweet", "retweet_with_comment", "like"])

  dummies = pd.get_dummies(
      df, 
      columns=['language', 'engaged_with_user_is_verified', 'enaging_user_is_verified', 'engagee_follows_engager', 'present_media', 'tweet_type'],
      prefix=['language', 'engaged_with_user_is_verified', 'enaging_user_is_verified', 'engagee_follows_engager', 'present_media', 'tweet_type'],
      prefix_sep="_",
      dummy_na=False,
      drop_first=False
  )
  return dummies

In [0]:
dummies = encoding(df)

In [0]:
def featurize(dummies):
  pca = decomposition.PCA(n_components=2)
  pca.fit(dummies)
  X_PCA = pca.transform(dummies)
  return X_PCA

In [0]:
X_PCA = featurize(dummies)

In [0]:
y_like = df['like']
y_reply = df['reply']
y_retweet_with_comment = df['retweet_with_comment']
y_retweet = df['retweet']

In [0]:
from xgboost import XGBClassifier
retweet = XGBClassifier()
retweet.fit(X_PCA, y_retweet)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
pickle.dump(retweet, open("retweet.pickle.dat", "wb"))